In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyarrow as pa
import pyarrow.parquet as pq
import geopandas as gpd
import seaborn as sns

#Packages
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import numpy as np
from scipy import stats
pd.set_option('display.max_columns', None)

from scipy.stats import skew, kurtosis

import pygris
import shapely

In [2]:
df = pd.read_parquet("C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/FimaNfipClaims.parquet.gzip")

In [51]:
df.shape[0]

2584242

In [3]:
df_copy = df.copy()

In [4]:
# Define bins and labels for yearOfLoss_1990_2021
bins_1980_2021 = [df_copy['yearOfLoss'].min(), 2000, 2010, 2020, df_copy['yearOfLoss'].max() + 1]
labels_1980_2021 = [1990, 2000, 2010, 2020]

df_copy['yearOfLoss_1990_2021'] = pd.cut(df_copy['yearOfLoss'], bins=bins_1980_2021, labels=labels_1980_2021, right=False).astype(int)

In [3]:
df_geographic_unique = df[['state', 'reportedZipCode', 'countyCode', 'censusTract', 'censusBlockGroupFips', 'latitude', 'longitude', 'yearOfLoss']].drop_duplicates()

In [4]:
df_geographic_unique = df_geographic_unique.dropna(subset=['latitude', 'censusBlockGroupFips'])

In [5]:
df_geographic_unique['censusBlockGroupFips'] = [str(int(float(i))) for i in df_geographic_unique['censusBlockGroupFips']]
df_geographic_unique['censusBlockGroupFips'] = [censusBG.zfill(12) for censusBG in df_geographic_unique['censusBlockGroupFips']]

In [6]:
# Define bins and labels for yearOfLoss_1980_2020
bins_1980_2020 = [df_geographic_unique['yearOfLoss'].min(), 1990, 2000, 2010, 2020, df_geographic_unique['yearOfLoss'].max() + 1]
labels_1980_2020 = [1980, 1990, 2000, 2010, 2020]

df_geographic_unique['yearOfLoss_1980_2020'] = pd.cut(df_geographic_unique['yearOfLoss'], bins=bins_1980_2020, labels=labels_1980_2020, right=False).astype(int)

In [ ]:
# states = pygris.states()
# states_list = sorted(states['STUSPS'].unique())

In [ ]:
# BG_list_2000 = []

# for i in states_list:
#     BG_list_2000.append(pygris.block_groups(state = i, year = 2000))

In [7]:
BG_list_df2000 = pygris.block_groups(state = 'NC', year = 2000)

Using FIPS code '37' for input 'NC'


In [ ]:
# BG_list_1990 = []

# for i in states_list:
#     BG_list_1990.append(pygris.block_groups(state = i, year = 1990, cb = True))

In [8]:
BG_list_df1990 = pygris.block_groups(state = 'NC', year = 1990, cb = True)

Using FIPS code '37' for input 'NC'


In [ ]:
# BG_list_df2000 = pd.concat(BG_list_2000, ignore_index=True)
# BG_list_df1990 = pd.concat(BG_list_1990, ignore_index=True)

In [9]:
BG_list_df2000 = BG_list_df2000[['BKGPIDFP00', 'geometry']]
BG_list_df2000['year'] = 2000

BG_list_df2000 = BG_list_df2000.rename(columns={"BKGPIDFP00": "GEOID"})

BG_list_df1990 = BG_list_df1990[['GEOID', 'geometry']]
BG_list_df1990['year'] = 1990

#### Checking for 2000...

In [ ]:
# df_geographic_unique2000 = df_geographic_unique[df_geographic_unique['yearOfLoss_1980_2020']==2000]

In [10]:
df_geographic_unique2000 = df_geographic_unique[(df_geographic_unique['yearOfLoss_1980_2020']==2000) & (df_geographic_unique['state']=='NC')]

In [11]:
# Read the parquet file
df_read = pd.read_parquet("C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/lat_long_geometry.parquet.gzip")

# Convert the WKT strings back to geometries
lat_long_df = gpd.GeoDataFrame(df_read, geometry=df_read['geometry'].apply(lambda x: shapely.wkt.loads(x)))

In [12]:
lat_long_df.rename(columns={'geometry': 'geometry_lat_long'}, inplace=True)

In [13]:
# Setting the multi-index on lat_long_df
lat_long_df.set_index(['latitude', 'longitude'], inplace=True)

# Mapping the values
df_geographic_unique2000['geometry_lat_long'] = df_geographic_unique2000.set_index(['latitude', 'longitude']).index.map(lat_long_df['geometry_lat_long'])

# Resetting the index of lat_long_df (optional, but good practice)
lat_long_df.reset_index(inplace=True)

C:\Users\Asus\AppData\Local\Temp\ipykernel_30356\408559743.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_geographic_unique2000['geometry_lat_long'] = df_geographic_unique2000.set_index(['latitude', 'longitude']).index.map(lat_long_df['geometry_lat_long'])


In [14]:
import geopandas as gpd
from shapely.geometry import Polygon
from itertools import product
import warnings
warnings.filterwarnings("ignore")

In [18]:
df_geographic_unique2000_lat_long = df_geographic_unique2000[['latitude', 'longitude', 'geometry_lat_long']].drop_duplicates()

In [20]:
# Convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(BG_list_df2000, geometry='geometry')

# Set the current CRS to WGS 84 (assuming your data is in latitude and longitude)
gdf.crs = "EPSG:4326"

# Project the GeoDataFrame to a suitable CRS (Web Mercator in this case)
gdf_projected = gdf.to_crs(epsg=3857)

# Compute the area for each geometry in square kilometers
gdf_projected['area_km2'] = gdf_projected.geometry.area / 1e6

# Filter out geometries with area less than 152.1611272703287 square kilometers
filtered_gdf = gdf_projected[gdf_projected['area_km2'] >= 152.16]

# If you want the filtered data in the original CRS, you can do the following:
BG_list_df2000_filtered = filtered_gdf.to_crs(epsg=4326)

In [55]:
# Create an empty GeoDataFrame to store the intersection results
new_unit_df2000 = gpd.GeoDataFrame(columns=['MATCHED_censusBlockGroupFips', 'latitude', 'longitude', 'geometry_BG',
                                        'geometry_lat_long', 'new_geometry'])

# Iterate through each row in BG_df_2000 and each row in lat_long_df to find intersections
for idx_unit_data, row_unit_data in df_geographic_unique2000_lat_long.iterrows():
    for idx_unit_shape, row_unit_shape in BG_list_df2000_filtered.iterrows():

        bg_id2 = row_unit_shape['GEOID']
        bg_geometry = row_unit_shape['geometry']
        lat_long_geometry = row_unit_data['geometry_lat_long']
        lat = row_unit_data['latitude']
        long = row_unit_data['longitude']
    
        # Compute intersection geometry
        intersection_geometry = bg_geometry.intersection(lat_long_geometry)
        
        intersection_gseries = gpd.GeoSeries([intersection_geometry], crs="EPSG:4326")
        intersection_gseries_projected = intersection_gseries.to_crs(epsg=3857)
        area_m2 = intersection_gseries_projected.iloc[0].area
        
        if area_m2 / 1e6 >= 100.1611272703287:
    
            new_unit_df2000 = pd.concat([new_unit_df2000, pd.DataFrame({
                'MATCHED_censusBlockGroupFips': [bg_id2],
                'latitude': [lat],
                'longitude': [long],
                'geometry_BG': [bg_geometry],
                'geometry_lat_long': [lat_long_geometry],
                'new_geometry': [intersection_geometry]
            })], ignore_index=True)

In [57]:
new_unit_df2000.shape[0]

8

In [58]:
# Setting the multi-index on filtered_df
new_unit_df2000.set_index(['latitude', 'longitude'], inplace=True)

# Mapping values one by one
df_geographic_unique2000['MATCHED_censusBlockGroupFips'] = df_geographic_unique2000.set_index(['latitude', 'longitude']).index.map(new_unit_df2000['MATCHED_censusBlockGroupFips'])
df_geographic_unique2000['geometry_BG'] = df_geographic_unique2000.set_index(['latitude', 'longitude']).index.map(new_unit_df2000['geometry_BG'])
df_geographic_unique2000['new_geometry'] = df_geographic_unique2000.set_index(['latitude', 'longitude']).index.map(new_unit_df2000['new_geometry'])

# Assuming lat_long_df should be df_geographic_unique2000 based on context
new_unit_df2000.reset_index(inplace=True)

In [59]:
df_geographic_unique2000.head()

,state,reportedZipCode,countyCode,censusTract,censusBlockGroupFips,latitude,longitude,yearOfLoss,yearOfLoss_1980_2020,geometry_lat_long,MATCHED_censusBlockGroupFips,geometry_BG,new_geometry
31,NC,28646.0,37011.0,3.701193e+10,370119302003,36.0,-81.9,2004,2000,"POLYGON ((-81.95000 35.95000, -81.85000 35.950...",NaN,NaN,NaN
90,NC,27920.0,37055.0,3.705597e+10,370559705022,35.3,-75.5,2003,2000,"POLYGON ((-75.55000 35.25000, -75.45000 35.250...",NaN,NaN,NaN
185,NC,27949.0,37055.0,3.705597e+10,370559701022,36.1,-75.7,2003,2000,"POLYGON ((-75.75000 36.05000, -75.65000 36.050...",NaN,NaN,NaN
235,NC,27927.0,37053.0,3.705311e+10,370531101012,36.4,-75.8,2006,2000,"POLYGON ((-75.85000 36.35000, -75.75000 36.350...",NaN,NaN,NaN
256,NC,27909.0,37139.0,3.713996e+10,371399605021,36.3,-76.2,2006,2000,"POLYGON ((-76.25000 36.25000, -76.15000 36.250...",NaN,NaN,NaN


In [61]:
filtered_df = df_geographic_unique2000[df_geographic_unique2000['MATCHED_censusBlockGroupFips'].notna()]

In [66]:
filtered_df[filtered_df['censusBlockGroupFips']!=filtered_df['MATCHED_censusBlockGroupFips']]

,state,reportedZipCode,countyCode,censusTract,censusBlockGroupFips,latitude,longitude,yearOfLoss,yearOfLoss_1980_2020,geometry_lat_long,MATCHED_censusBlockGroupFips,geometry_BG,new_geometry
5407,NC,27978.0,37055.0,3.705597e+10,370559705011,35.7,-75.7,2003,2000,"POLYGON ((-75.75000 35.65000, -75.65000 35.650...",370559705004,"POLYGON ((-75.59184199999999 35.758117, -75.59...","POLYGON ((-75.75 35.75, -75.65 35.75, -75.65 3..."
14690,NC,27824.0,37095.0,3.709592e+10,370959201003,35.5,-76.0,2003,2000,"POLYGON ((-76.05000 35.45000, -75.95000 35.450...",370959801001,"POLYGON ((-76.211888 35.46288299999999, -76.21...","POLYGON ((-76.05 35.55, -75.95 35.55, -75.95 3..."
241614,NC,27978.0,37055.0,3.705597e+10,370559705011,35.7,-75.7,2004,2000,"POLYGON ((-75.75000 35.65000, -75.65000 35.650...",370559705004,"POLYGON ((-75.59184199999999 35.758117, -75.59...","POLYGON ((-75.75 35.75, -75.65 35.75, -75.65 3..."
403717,NC,27978.0,37055.0,3.705597e+10,370559705011,35.7,-75.7,2008,2000,"POLYGON ((-75.75000 35.65000, -75.65000 35.650...",370559705004,"POLYGON ((-75.59184199999999 35.758117, -75.59...","POLYGON ((-75.75 35.75, -75.65 35.75, -75.65 3..."
403718,NC,27978.0,37055.0,3.705597e+10,370559705011,35.7,-75.7,2009,2000,"POLYGON ((-75.75000 35.65000, -75.65000 35.650...",370559705004,"POLYGON ((-75.59184199999999 35.758117, -75.59...","POLYGON ((-75.75 35.75, -75.65 35.75, -75.65 3..."
550378,NC,27824.0,37095.0,3.709592e+10,370959201003,35.5,-76.0,2007,2000,"POLYGON ((-76.05000 35.45000, -75.95000 35.450...",370959801001,"POLYGON ((-76.211888 35.46288299999999, -76.21...","POLYGON ((-76.05 35.55, -75.95 35.55, -75.95 3..."
554301,NC,27978.0,37055.0,3.705597e+10,370559705011,35.7,-75.8,2003,2000,"POLYGON ((-75.85000 35.65000, -75.75000 35.650...",370559705004,"POLYGON ((-75.59184199999999 35.758117, -75.59...","POLYGON ((-75.85 35.75, -75.75 35.75, -75.75 3..."
624424,NC,27824.0,37095.0,3.709592e+10,370959201003,35.5,-76.1,2003,2000,"POLYGON ((-76.15000 35.45000, -76.05000 35.450...",370959801001,"POLYGON ((-76.211888 35.46288299999999, -76.21...","POLYGON ((-76.14999999999999 35.55, -76.05 35...."
625303,NC,27824.0,37095.0,3.709592e+10,370959201003,35.5,-76.0,2004,2000,"POLYGON ((-76.05000 35.45000, -75.95000 35.450...",370959801001,"POLYGON ((-76.211888 35.46288299999999, -76.21...","POLYGON ((-76.05 35.55, -75.95 35.55, -75.95 3..."
626412,NC,27824.0,37095.0,3.709592e+10,370959201003,35.6,-76.1,2003,2000,"POLYGON ((-76.15000 35.55000, -76.05000 35.550...",370959801001,"POLYGON ((-76.211888 35.46288299999999, -76.21...","POLYGON ((-76.14999999999999 35.65, -76.05 35...."


In [67]:
filtered_df.to_csv('C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/BG_analysis.csv')

#### Checking for 1990

In [ ]:
# df_geographic_unique1990 = df_geographic_unique[df_geographic_unique['yearOfLoss_1980_2020']==1990]

In [ ]:
df_geographic_unique1990= df_geographic_unique[(df_geographic_unique['yearOfLoss_1980_2020']==1990) & (df_geographic_unique['state']=='NC')]

In [ ]:
# Setting the multi-index on lat_long_df
lat_long_df.set_index(['latitude', 'longitude'], inplace=True)

# Mapping the values
df_geographic_unique1990['geometry_lat_long'] = df_geographic_unique1990.set_index(['latitude', 'longitude']).index.map(lat_long_df['geometry_lat_long'])

# Resetting the index of lat_long_df (optional, but good practice)
lat_long_df.reset_index(inplace=True)

In [ ]:
# Create an empty GeoDataFrame to store the intersection results
new_unit_df1990 = gpd.GeoDataFrame(columns=['reportedZipCode', 'countyCode', 'censusTract',
                                       'censusBlockGroupFips', 'MATCHED_censusBlockGroupFips', 'latitude', 'longitude', 'year', 'state', 'geometry_BG',
                                        'geometry_lat_long', 'new_geometry'])

# Iterate through each row in BG_df_2000 and each row in lat_long_df to find intersections
for idx_unit_data, row_unit_data in df_geographic_unique2000.iterrows():
    for idx_unit_shape, row_unit_shape in BG_list_df2000.iterrows():
        year = row_unit_data['yearOfLoss']
        year2 = row_unit_data['yearOfLoss_1980_2020']
        bg_id = row_unit_data['censusBlockGroupFips']
        bg_id2 = row_unit_shape['GEOID']
        bg_geometry = row_unit_shape['geometry']
        tract_id = row_unit_data['censusTract']
        county_id = row_unit_data['countyCode']
        state = row_unit_data['state']
        lat_long_geometry = row_unit_data['geometry_lat_long']
        lat = row_unit_data['latitude']
        long = row_unit_data['longitude']
        zipcode = row_unit_data['reportedZipCode']
    
        # Compute intersection geometry
        intersection_geometry = bg_geometry.intersection(lat_long_geometry)
    
        new_unit_df1990 = pd.concat([new_unit_df1990, pd.DataFrame({
            'reportedZipCode': [zipcode],
            'countyCode': [county_id],
            'censusTract': [tract_id],
            'censusBlockGroupFips': [bg_id],
            'MATCHED_censusBlockGroupFips': [bg_id2],
            'latitude': [lat],
            'longitude': [long],
            'year': [year],
            'decade': [year2],
            'state': [state],
            'geometry_BG': [bg_geometry],
            'geometry_lat_long': [lat_long_geometry],
            'new_geometry': [intersection_geometry]
        })], ignore_index=True)

## Checking if 2010 BG shapefiles match for the entire dataset

In [3]:
states = pygris.states()
states_list = sorted(states['STUSPS'].unique())

Using the default year of 2021


In [52]:
BG_list_2010 = []

for i in states_list:
    BG_list_2010.append(pygris.block_groups(state = i, year = 2010))

Using FIPS code '02' for input 'AK'
Using FIPS code '01' for input 'AL'
Using FIPS code '05' for input 'AR'
Using FIPS code '60' for input 'AS'
Using FIPS code '04' for input 'AZ'
Using FIPS code '06' for input 'CA'
Using FIPS code '08' for input 'CO'
Using FIPS code '09' for input 'CT'
Using FIPS code '11' for input 'DC'
Using FIPS code '10' for input 'DE'
Using FIPS code '12' for input 'FL'
Using FIPS code '13' for input 'GA'
Using FIPS code '66' for input 'GU'
Using FIPS code '15' for input 'HI'
Using FIPS code '19' for input 'IA'
Using FIPS code '16' for input 'ID'
Using FIPS code '17' for input 'IL'
Using FIPS code '18' for input 'IN'
Using FIPS code '20' for input 'KS'
Using FIPS code '21' for input 'KY'
Using FIPS code '22' for input 'LA'
Using FIPS code '25' for input 'MA'
Using FIPS code '24' for input 'MD'
Using FIPS code '23' for input 'ME'
Using FIPS code '26' for input 'MI'
Using FIPS code '27' for input 'MN'
Using FIPS code '29' for input 'MO'
Using FIPS code '69' for inp

In [53]:
BG_list_2011 = []

for i in states_list:
    BG_list_2011.append(pygris.block_groups(state = i, year = 2011))

Using FIPS code '02' for input 'AK'
Using FIPS code '01' for input 'AL'
Using FIPS code '05' for input 'AR'
Using FIPS code '60' for input 'AS'
Using FIPS code '04' for input 'AZ'
Using FIPS code '06' for input 'CA'
Using FIPS code '08' for input 'CO'
Using FIPS code '09' for input 'CT'
Using FIPS code '11' for input 'DC'
Using FIPS code '10' for input 'DE'
Using FIPS code '12' for input 'FL'
Using FIPS code '13' for input 'GA'
Using FIPS code '66' for input 'GU'
Using FIPS code '15' for input 'HI'
Using FIPS code '19' for input 'IA'
Using FIPS code '16' for input 'ID'
Using FIPS code '17' for input 'IL'
Using FIPS code '18' for input 'IN'
Using FIPS code '20' for input 'KS'
Using FIPS code '21' for input 'KY'
Using FIPS code '22' for input 'LA'
Using FIPS code '25' for input 'MA'
Using FIPS code '24' for input 'MD'
Using FIPS code '23' for input 'ME'
Using FIPS code '26' for input 'MI'
Using FIPS code '27' for input 'MN'
Using FIPS code '29' for input 'MO'
Using FIPS code '69' for inp

In [54]:
BG_list_2012 = []

for i in states_list:
    BG_list_2012.append(pygris.block_groups(state = i, year = 2012))

Using FIPS code '02' for input 'AK'
Using FIPS code '01' for input 'AL'
Using FIPS code '05' for input 'AR'
Using FIPS code '60' for input 'AS'
Using FIPS code '04' for input 'AZ'
Using FIPS code '06' for input 'CA'
Using FIPS code '08' for input 'CO'
Using FIPS code '09' for input 'CT'
Using FIPS code '11' for input 'DC'
Using FIPS code '10' for input 'DE'
Using FIPS code '12' for input 'FL'
Using FIPS code '13' for input 'GA'
Using FIPS code '66' for input 'GU'
Using FIPS code '15' for input 'HI'
Using FIPS code '19' for input 'IA'
Using FIPS code '16' for input 'ID'
Using FIPS code '17' for input 'IL'
Using FIPS code '18' for input 'IN'
Using FIPS code '20' for input 'KS'
Using FIPS code '21' for input 'KY'
Using FIPS code '22' for input 'LA'
Using FIPS code '25' for input 'MA'
Using FIPS code '24' for input 'MD'
Using FIPS code '23' for input 'ME'
Using FIPS code '26' for input 'MI'
Using FIPS code '27' for input 'MN'
Using FIPS code '29' for input 'MO'
Using FIPS code '69' for inp

In [55]:
BG_list_2018 = []

for i in states_list:
    BG_list_2018.append(pygris.block_groups(state = i, year = 2018))

Using FIPS code '02' for input 'AK'
Using FIPS code '01' for input 'AL'
Using FIPS code '05' for input 'AR'
Using FIPS code '60' for input 'AS'
Using FIPS code '04' for input 'AZ'
Using FIPS code '06' for input 'CA'
Using FIPS code '08' for input 'CO'
Using FIPS code '09' for input 'CT'
Using FIPS code '11' for input 'DC'
Using FIPS code '10' for input 'DE'
Using FIPS code '12' for input 'FL'
Using FIPS code '13' for input 'GA'
Using FIPS code '66' for input 'GU'
Using FIPS code '15' for input 'HI'
Using FIPS code '19' for input 'IA'
Using FIPS code '16' for input 'ID'
Using FIPS code '17' for input 'IL'
Using FIPS code '18' for input 'IN'
Using FIPS code '20' for input 'KS'
Using FIPS code '21' for input 'KY'
Using FIPS code '22' for input 'LA'
Using FIPS code '25' for input 'MA'
Using FIPS code '24' for input 'MD'
Using FIPS code '23' for input 'ME'
Using FIPS code '26' for input 'MI'
Using FIPS code '27' for input 'MN'
Using FIPS code '29' for input 'MO'
Using FIPS code '69' for inp

In [83]:
temp2018 = pd.concat(BG_list_2018, ignore_index=True)
temp2012 = pd.concat(BG_list_2012, ignore_index=True)
temp2011 = pd.concat(BG_list_2011, ignore_index=True)
temp2010 = pd.concat(BG_list_2010, ignore_index=True)

In [84]:
temp2018 = temp2018[['GEOID', 'geometry']]
temp2018['year'] = 2010

temp2012 = temp2012[['GEOID', 'geometry']]
temp2012['year'] = 2010

temp2011 = temp2011[['GEOID', 'geometry']]
temp2011['year'] = 2010

temp2010 = temp2010[['GEOID10', 'geometry']]
temp2010['year'] = 2010

temp2010 = temp2010.rename(columns={"GEOID10": "GEOID"})

In [159]:
list_censusBG_2 = []

In [172]:
list_censusBG = temp2018['GEOID'].dropna().drop_duplicates().astype(str).tolist()

# add preceding 0 to make 4-digit zip codes into 5-digit
list_censusBG = [censusBG.zfill(12) for censusBG in list_censusBG]

In [173]:
len(list_censusBG)

220740

In [174]:
list_censusBG_2 = list_censusBG_2 + list_censusBG

In [175]:
len(list_censusBG_2)

441482

In [176]:
list_censusBG_2 = list(set(list_censusBG_2))

In [177]:
len(list_censusBG_2)

220814

In [178]:
#CensusBG list for NC

list_censusBG = df['censusBlockGroupFips'].dropna().drop_duplicates().tolist()

list_censusBG = [str(int(float(i))) for i in list_censusBG]

# add preceding 0 to make 4-digit zip codes into 5-digit
list_censusBG = [censusBG.zfill(12) for censusBG in list_censusBG]

In [179]:
# Convert both lists to sets
set_bg = set(list_censusBG)
set_bg_2 = set(list_censusBG_2)

# Find the intersection of the two sets
common_bg = set_bg.intersection(set_bg_2)

# Print the number of common zip codes
print("Number of common BG:", len(common_bg))

Number of common BG: 105940


In [180]:
len(common_bg)/len(list_censusBG)

0.9920776131702658

In [181]:
len(common_bg)

105940

In [182]:
len(list_censusBG)-len(common_bg)

846

In [171]:
list_2010_BG = common_bg

In [53]:
df_copy = df.dropna(subset=['censusBlockGroupFips'])

In [55]:
df_copy['censusBlockGroupFips'] = [str(int(float(i))) for i in df_copy['censusBlockGroupFips']]
df_copy['censusBlockGroupFips'] = [censusBG.zfill(12) for censusBG in df_copy['censusBlockGroupFips']]

C:\Users\Asus\AppData\Local\Temp\ipykernel_21236\361452524.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_copy['censusBlockGroupFips'] = [str(int(float(i))) for i in df_copy['censusBlockGroupFips']]
C:\Users\Asus\AppData\Local\Temp\ipykernel_21236\361452524.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_copy['censusBlockGroupFips'] = [censusBG.zfill(12) for censusBG in df_copy['censusBlockGroupFips']]


In [74]:
df_test = df_copy[~df_copy['censusBlockGroupFips'].isin(common_bg)]

In [69]:
df_copy[df_copy['censusBlockGroupFips'].isin(common_bg)].shape[0]

2445442

In [66]:
df_copy[(~df_copy['censusBlockGroupFips'].isin(common_bg)) & (df_copy['state']=='NC')]

189

In [183]:
BG_list_df2018 = temp2018[temp2018['GEOID'].isin(common_bg)]
BG_list_df2012 = temp2012[temp2012['GEOID'].isin(common_bg)]
BG_list_df2011 = temp2011[temp2011['GEOID'].isin(common_bg)]
BG_list_df2010 = temp2010[temp2010['GEOID'].isin(common_bg)]

In [184]:
print(BG_list_df2018.shape[0])
print(BG_list_df2012.shape[0])
print(BG_list_df2011.shape[0])
print(BG_list_df2010.shape[0])

105901
105904
105909
105926


In [185]:
BG_df = pd.concat([BG_list_df2018, BG_list_df2010], ignore_index=True)

In [186]:
BG_df2 = BG_df.drop_duplicates()

In [187]:
BG_df2.shape[0]

211419

In [188]:
BG_df2 = BG_df2.drop_duplicates(subset=['GEOID', 'year'])

In [189]:
BG_df2.shape[0]

105940

### Some checks...

In [199]:
list_censusBG_check = BG_df2['GEOID'].dropna().drop_duplicates().astype(str).tolist()

# add preceding 0 to make 4-digit zip codes into 5-digit
list_censusBG_check = [censusBG.zfill(12) for censusBG in list_censusBG_check]

In [201]:
#CensusBG list

list_censusBG = df['censusBlockGroupFips'].dropna().drop_duplicates().tolist()

list_censusBG = [str(int(float(i))) for i in list_censusBG]

# add preceding 0 to make 4-digit zip codes into 5-digit
list_censusBG = [censusBG.zfill(12) for censusBG in list_censusBG]

In [202]:
# Convert both lists to sets
set_bg = set(list_censusBG)
set_bg_2 = set(list_censusBG_2)

# Find the intersection of the two sets
common_bg = set_bg.intersection(set_bg_2)

# Print the number of common zip codes
print("Number of common BG:", len(common_bg))

Number of common BG: 105940


In [203]:
len(list_censusBG)-len(common_bg)

846

In [207]:
sum(BG_df2['geometry_BG'].isna())

0

### Post 2020 Match Analysis

In [70]:
df_2020 = df_copy[df_copy['yearOfLoss']>=2020]

In [73]:
BG_list_2020 = []

for i in states_list:
    BG_list_2020.append(pygris.block_groups(state = i, year = 2020))

Using FIPS code '02' for input 'AK'
Using FIPS code '01' for input 'AL'
Using FIPS code '05' for input 'AR'
Using FIPS code '60' for input 'AS'
Using FIPS code '04' for input 'AZ'
Using FIPS code '06' for input 'CA'
Using FIPS code '08' for input 'CO'
Using FIPS code '09' for input 'CT'
Using FIPS code '11' for input 'DC'
Using FIPS code '10' for input 'DE'
Using FIPS code '12' for input 'FL'
Using FIPS code '13' for input 'GA'
Using FIPS code '66' for input 'GU'
Using FIPS code '15' for input 'HI'
Using FIPS code '19' for input 'IA'
Using FIPS code '16' for input 'ID'
Using FIPS code '17' for input 'IL'
Using FIPS code '18' for input 'IN'
Using FIPS code '20' for input 'KS'
Using FIPS code '21' for input 'KY'
Using FIPS code '22' for input 'LA'
Using FIPS code '25' for input 'MA'
Using FIPS code '24' for input 'MD'
Using FIPS code '23' for input 'ME'
Using FIPS code '26' for input 'MI'
Using FIPS code '27' for input 'MN'
Using FIPS code '29' for input 'MO'
Using FIPS code '69' for inp

In [75]:
temp2020 = pd.concat(BG_list_2020, ignore_index=True)

In [106]:
list_censusBG2 = temp2020['GEOID'].dropna().drop_duplicates().astype(str).tolist()

# add preceding 0 to make 4-digit zip codes into 5-digit
list_censusBG2 = [censusBG.zfill(12) for censusBG in list_censusBG2]

In [107]:
len(list_censusBG2)

242747

In [108]:
#CensusBG list for NC

list_censusBG = df_2020['censusBlockGroupFips'].dropna().drop_duplicates().tolist()

list_censusBG = [str(int(float(i))) for i in list_censusBG]

# add preceding 0 to make 4-digit zip codes into 5-digit
list_censusBG = [censusBG.zfill(12) for censusBG in list_censusBG]

In [109]:
# Convert both lists to sets
set_bg = set(list_censusBG)
set_bg_2 = set(list_censusBG2)

# Find the intersection of the two sets
common_bg = set_bg.intersection(set_bg_2)

# Print the number of common zip codes
print("Number of common BG:", len(common_bg))

Number of common BG: 17900


In [110]:
len(common_bg)/len(list_censusBG)

0.7374448976228731

In [111]:
len(common_bg)

17900

In [112]:
len(list_censusBG)-len(common_bg)

6373

In [ ]:
## Of the remaining censusBlockGroupFips which didn't match in 2010 shapefile

In [113]:
#CensusBG list for NC

list_censusBG = df_test['censusBlockGroupFips'].dropna().drop_duplicates().tolist()

list_censusBG = [str(int(float(i))) for i in list_censusBG]

# add preceding 0 to make 4-digit zip codes into 5-digit
list_censusBG = [censusBG.zfill(12) for censusBG in list_censusBG]

In [117]:
# Convert both lists to sets
set_bg = set(list_censusBG)
set_bg_2 = set(list_censusBG2)

# Find the intersection of the two sets
common_bg = set_bg.intersection(set_bg_2)

# Print the number of common zip codes
print("Number of common BG:", len(common_bg))

Number of common BG: 416


In [118]:
len(common_bg)/len(list_censusBG)

0.491725768321513

In [119]:
len(common_bg)

416

In [120]:
len(list_censusBG)-len(common_bg)

430

In [121]:
df_test[df_test['state'].isin(['NC', 'WA'])].head()

,agricultureStructureIndicator,asOfDate,basementEnclosureCrawlspaceType,policyCount,crsClassificationCode,dateOfLoss,elevatedBuildingIndicator,elevationCertificateIndicator,elevationDifference,baseFloodElevation,ratedFloodZone,houseWorship,locationOfContents,lowestAdjacentGrade,lowestFloorElevation,numberOfFloorsInTheInsuredBuilding,nonProfitIndicator,obstructionType,occupancyType,originalConstructionDate,originalNBDate,amountPaidOnBuildingClaim,amountPaidOnContentsClaim,amountPaidOnIncreasedCostOfComplianceClaim,postFIRMConstructionIndicator,rateMethod,smallBusinessIndicatorBuilding,totalBuildingInsuranceCoverage,totalContentsInsuranceCoverage,yearOfLoss,primaryResidenceIndicator,buildingDamageAmount,buildingDeductibleCode,netBuildingPaymentAmount,buildingPropertyValue,causeOfDamage,condominiumCoverageTypeCode,contentsDamageAmount,contentsDeductibleCode,netContentsPaymentAmount,contentsPropertyValue,disasterAssistanceCoverageRequired,eventDesignationNumber,ficoNumber,floodCharacteristicsIndicator,floodWaterDuration,floodproofedIndicator,floodEvent,iccCoverage,netIccPaymentAmount,nfipRatedCommunityNumber,nfipCommunityNumberCurrent,nfipCommunityName,nonPaymentReasonContents,nonPaymentReasonBuilding,numberOfUnits,buildingReplacementCost,contentsReplacementCost,replacementCostBasis,stateOwnedIndicator,waterDepth,floodZoneCurrent,buildingDescriptionCode,rentalPropertyIndicator,state,reportedCity,reportedZipCode,countyCode,censusTract,censusBlockGroupFips,latitude,longitude,id
2169,0,2019-09-19T06:12:43.388Z,2.0,1,NaN,1996-09-05T00:00:00.000Z,1,nan,NaN,NaN,X,0,2.0,NaN,NaN,3.0,0,60.0,3.0,1984-01-01T00:00:00.000Z,1995-12-13T00:00:00.000Z,NaN,NaN,NaN,1,1,0,20000.0,15000.0,1996,0,NaN,0,0.00,NaN,1,U,NaN,0,0.0,NaN,0.0,None,592.0,NaN,0.0,0,Hurricane Fran,NaN,0.0,370170.0,None,"KURE BEACH, TOWN OF",97.0,97.0,1.0,NaN,NaN,A,0,0.0,None,NaN,0,NC,Currently Unavailable,28449.0,37129.0,3.712901e+10,371290122004,34.0,-77.9,1fa7289a-b432-42ad-8af3-bba5bd65b697
17637,0,2020-01-22T16:55:53.194Z,NaN,1,6.0,2003-09-18T00:00:00.000Z,0,nan,NaN,NaN,X,0,NaN,NaN,NaN,2.0,0,NaN,1.0,1989-07-15T00:00:00.000Z,1993-02-03T00:00:00.000Z,4641.55,0.0,0.0,1,7,0,150000.0,38000.0,2003,1,5141.0,0,4641.55,139500.0,1,N,472.0,0,0.0,0.0,0.0,None,639.0,NaN,0.0,0,Hurricane Isabel,20000.0,0.0,370439.0,None,"KITTY HAWK, TOWN OF",1.0,NaN,1.0,150000.0,0.0,A,0,0.0,None,NaN,0,NC,Currently Unavailable,27949.0,37055.0,3.705597e+10,370559701002,36.1,-75.7,074b8629-bd2f-4f9b-8228-f993529a57fb
91383,0,2020-01-22T16:55:53.194Z,NaN,10,8.0,1999-09-15T00:00:00.000Z,0,nan,3.0,14.0,AE,0,NaN,16.6,16.6,3.0,0,NaN,3.0,1985-07-01T00:00:00.000Z,1993-05-02T00:00:00.000Z,282.75,0.0,0.0,1,1,0,625000.0,0.0,1999,0,785.0,0,282.75,667800.0,4,H,NaN,None,0.0,NaN,0.0,None,626.0,NaN,0.0,0,Hurricane Floyd,15000.0,0.0,375357.0,375357,"OCEAN ISLE BEACH, TOWN OF",NaN,NaN,10.0,718000.0,NaN,A,0,0.0,VE,1.0,0,NC,Currently Unavailable,28459.0,37019.0,3.701902e+10,370190205033,33.9,-78.5,b85f8c99-8673-46ac-8abe-b138c39134e6
91873,0,2019-09-19T13:45:58.425Z,NaN,1,NaN,1996-09-05T00:00:00.000Z,0,nan,NaN,NaN,X,0,NaN,NaN,NaN,1.0,0,10.0,3.0,1981-01-06T00:00:00.000Z,1994-03-19T00:00:00.000Z,9228.45,0.0,0.0,0,1,0,250000.0,0.0,1996,0,250000.0,1,9228.45,300000.0,1,N,100000.0,None,0.0,0.0,0.0,None,592.0,NaN,0.0,0,Hurricane Fran,NaN,0.0,370170.0,None,"KURE BEACH, TOWN OF",NaN,NaN,1.0,300000.0,0.0,A,0,24.0,None,NaN,0,NC,Currently Unavailable,28449.0,37129.0,3.712901e+10,371290122004,34.0,-77.9,f2e6ee86-0db6-42cf-a726-903ececbdc9a
92010,0,2019-10-19T03:42:08.849Z,NaN,1,NaN,1996-07-12T00:00:00.000Z,0,nan,NaN,NaN,X,0,4.0,NaN,NaN,2.0,0,10.0,4.0,1981-01-06T00:00:00.000Z,1994-06-16T00:00:00.000Z,3780.55,0.0,0.0,0,1,0,200000.0,0.0,1996,0,4781.0,1,3780.55,273240.0,1,N,NaN,None,0.0,NaN,0.0,None,590.0,NaN,0.0,0,Hurricane Bertha,NaN,0.0,370170.0,None,"KURE BEACH, TOWN OF",97.0,NaN,1.0,0.0,NaN,A,0,0.0,None,NaN,0,NC,Currently Unavailable,28449.0,37129.0,3.712901e+10,371290122004,34.0,-77.9,33afe57a-9966-41fd-8616-e2d34e0f795f


In [129]:
df_temp = df_test[df_test['censusBlockGroupFips'].isin(common_bg)]

In [130]:
df_temp['yearOfLoss'].value_counts()

2022    237
2017    133
2021     76
2020     58
2016     53
2012     45
2023     29
2018     26
2005     23
2019     20
2011     15
2004     10
2015      8
2008      7
2010      6
2003      5
2014      3
2000      3
1993      3
2006      3
1998      2
1996      1
1992      1
1991      1
1990      1
1997      1
1995      1
2002      1
2009      1
2001      1
2007      1
Name: yearOfLoss, dtype: int64

### 6. censusBlockGroupFips

In [152]:
test = df[df['censusBlockGroupFips'].notna()]

In [153]:
print('Total observations available:', test.shape[0])
print('Observations lost:', df.shape[0]-test.shape[0])
print('Percentage NA:', (df.shape[0]-test.shape[0])*100/df.shape[0])
print('Total unique censusBlockGroup:', test.drop_duplicates(subset=['censusBlockGroupFips']).shape[0])

Total observations available: 2447264
Observations lost: 136978
Percentage NA: 5.300509781978623
Total unique censusBlockGroup: 106786


##### Now we can calculate for each decade how many of the census tract have shapefiles with geometry attribute.

In [190]:
df_geographic_unique = df[['censusBlockGroupFips', 'yearOfLoss']].drop_duplicates()
df_geographic_unique = df_geographic_unique.dropna(subset=['censusBlockGroupFips'])

In [191]:
df_geographic_unique['censusBlockGroupFips'] = [str(int(float(i))) for i in df_geographic_unique['censusBlockGroupFips']]
df_geographic_unique['censusBlockGroupFips'] = [censusBG.zfill(12) for censusBG in df_geographic_unique['censusBlockGroupFips']]

In [192]:
# Define bins and labels for yearOfLoss_1990_2021
bins_1990_2021 = [df_geographic_unique['yearOfLoss'].min(), 2000, 2010, 2020, df_geographic_unique['yearOfLoss'].max() + 1]
labels_1990_2021 = [1990, 2000, 2010, 2020]

df_geographic_unique['yearOfLoss_1990_2021'] = pd.cut(df_geographic_unique['yearOfLoss'], bins=bins_1990_2021, labels=labels_1990_2021, right=False).astype(int)

df_geographic_unique = df_geographic_unique.drop(columns='yearOfLoss')

In [193]:
df_geographic_unique = df_geographic_unique.drop_duplicates()

In [194]:
BG_df2.rename(columns={'geometry': 'geometry_BG'}, inplace=True)

In [195]:
BG_df2.set_index(['GEOID'], inplace=True)
df_geographic_unique['geometry_BG'] = df_geographic_unique.set_index(['censusBlockGroupFips']).index.map(BG_df2['geometry_BG'])

# Find rows with NaN values 
na_rows = df_geographic_unique['geometry_BG'].isna()

BG_df2.reset_index(inplace=True)

In [198]:
sum(na_rows)

1024

In [211]:
test = df_geographic_unique.loc[na_rows, ]
print('Total missing:', test[['censusBlockGroupFips']].drop_duplicates().shape[0])
print('Number of censusBlockGroupFips per decade without a matching geometry')
print(test.drop_duplicates(subset=['censusBlockGroupFips', 'yearOfLoss_1990_2021'])['yearOfLoss_1990_2021'].value_counts())

Total missing: 846
Number of censusBlockGroupFips per decade without a matching geometry
2000    311
2010    266
1990    224
2020    223
Name: yearOfLoss_1990_2021, dtype: int64


In [205]:
df_test = df_copy[df_copy['censusBlockGroupFips'].notna()]

df_test['censusBlockGroupFips'] = [str(int(float(i))) for i in df_test['censusBlockGroupFips']]
df_test['censusBlockGroupFips'] = [censusBG.zfill(12) for censusBG in df_test['censusBlockGroupFips']]

C:\Users\Asus\AppData\Local\Temp\ipykernel_21236\889359098.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['censusBlockGroupFips'] = [str(int(float(i))) for i in df_test['censusBlockGroupFips']]
C:\Users\Asus\AppData\Local\Temp\ipykernel_21236\889359098.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['censusBlockGroupFips'] = [censusBG.zfill(12) for censusBG in df_test['censusBlockGroupFips']]


In [206]:
print('Value_counts of censusBlockGroupFips per decade:\n', df_test.drop_duplicates(subset=['censusBlockGroupFips', 'yearOfLoss_1990_2021'])['yearOfLoss_1990_2021'].value_counts())

Value_counts of censusBlockGroupFips per decade:
 1990    71925
2010    61534
2000    52227
2020    24273
Name: yearOfLoss_1990_2021, dtype: int64


In [207]:
test.set_index(['censusBlockGroupFips', 'yearOfLoss_1990_2021'], inplace=True)
df_test.set_index(['censusBlockGroupFips', 'yearOfLoss_1990_2021'], inplace=True)

# Select rows of df_copy that have indices found in test
matching_rows = df_test.loc[df_test.index.isin(test.index)]

matching_rows.reset_index(inplace=True)
test.reset_index(inplace=True)
df_test.reset_index(inplace=True)

print(f"There are {matching_rows.shape[0]} rows in df_copy with the same censusBlockGroupFips and yearOfLoss_1990_2021 as in test.")

There are 1822 rows in df_copy with the same censusBlockGroupFips and yearOfLoss_1990_2021 as in test.


In [208]:
matching_rows['yearOfLoss_1990_2021'].value_counts()

2000    547
1990    451
2010    424
2020    400
Name: yearOfLoss_1990_2021, dtype: int64

### Match Rate using 2020 shapefiles on the entire dataset...

In [5]:
states = pygris.states()
states_list = sorted(states['STUSPS'].unique())

Using the default year of 2021


In [6]:
BG_list_2020 = []

for i in states_list:
    BG_list_2020.append(pygris.block_groups(state = i, year = 2020))

Using FIPS code '02' for input 'AK'
Using FIPS code '01' for input 'AL'
Using FIPS code '05' for input 'AR'
Using FIPS code '60' for input 'AS'
Using FIPS code '04' for input 'AZ'
Using FIPS code '06' for input 'CA'
Using FIPS code '08' for input 'CO'
Using FIPS code '09' for input 'CT'
Using FIPS code '11' for input 'DC'
Using FIPS code '10' for input 'DE'
Using FIPS code '12' for input 'FL'
Using FIPS code '13' for input 'GA'
Using FIPS code '66' for input 'GU'
Using FIPS code '15' for input 'HI'
Using FIPS code '19' for input 'IA'
Using FIPS code '16' for input 'ID'
Using FIPS code '17' for input 'IL'
Using FIPS code '18' for input 'IN'
Using FIPS code '20' for input 'KS'
Using FIPS code '21' for input 'KY'
Using FIPS code '22' for input 'LA'
Using FIPS code '25' for input 'MA'
Using FIPS code '24' for input 'MD'
Using FIPS code '23' for input 'ME'
Using FIPS code '26' for input 'MI'
Using FIPS code '27' for input 'MN'
Using FIPS code '29' for input 'MO'
Using FIPS code '69' for inp

In [7]:
BG_list_2022 = []

for i in states_list:
    BG_list_2022.append(pygris.block_groups(state = i, year = 2022))

Using FIPS code '02' for input 'AK'
Using FIPS code '01' for input 'AL'
Using FIPS code '05' for input 'AR'
Using FIPS code '60' for input 'AS'
Using FIPS code '04' for input 'AZ'
Using FIPS code '06' for input 'CA'
Using FIPS code '08' for input 'CO'
Using FIPS code '09' for input 'CT'
Using FIPS code '11' for input 'DC'
Using FIPS code '10' for input 'DE'
Using FIPS code '12' for input 'FL'
Using FIPS code '13' for input 'GA'
Using FIPS code '66' for input 'GU'
Using FIPS code '15' for input 'HI'
Using FIPS code '19' for input 'IA'
Using FIPS code '16' for input 'ID'
Using FIPS code '17' for input 'IL'
Using FIPS code '18' for input 'IN'
Using FIPS code '20' for input 'KS'
Using FIPS code '21' for input 'KY'
Using FIPS code '22' for input 'LA'
Using FIPS code '25' for input 'MA'
Using FIPS code '24' for input 'MD'
Using FIPS code '23' for input 'ME'
Using FIPS code '26' for input 'MI'
Using FIPS code '27' for input 'MN'
Using FIPS code '29' for input 'MO'
Using FIPS code '69' for inp

In [8]:
temp2020 = pd.concat(BG_list_2020, ignore_index=True)
temp2022 = pd.concat(BG_list_2022, ignore_index=True)

In [9]:
temp2020 = temp2020[['GEOID', 'geometry']]
temp2020['year'] = 2020

temp2022 = temp2022[['GEOID', 'geometry']]
temp2022['year'] = 2020

In [10]:
list_censusBG_2 = []

In [22]:
list_censusBG = temp2022['GEOID'].dropna().drop_duplicates().astype(str).tolist()

# add preceding 0 to make 4-digit zip codes into 5-digit
list_censusBG = [censusBG.zfill(12) for censusBG in list_censusBG]

In [23]:
len(list_censusBG)

242748

In [24]:
list_censusBG_2 = list_censusBG_2 + list_censusBG

In [25]:
len(list_censusBG_2)

485495

In [26]:
list_censusBG_2 = list(set(list_censusBG_2))

In [27]:
len(list_censusBG_2)

245464

In [28]:
#CensusBG list for NC

list_censusBG = df['censusBlockGroupFips'].dropna().drop_duplicates().tolist()

list_censusBG = [str(int(float(i))) for i in list_censusBG]

# add preceding 0 to make 4-digit zip codes into 5-digit
list_censusBG = [censusBG.zfill(12) for censusBG in list_censusBG]

In [29]:
# Convert both lists to sets
set_bg = set(list_censusBG)
set_bg_2 = set(list_censusBG_2)

# Find the intersection of the two sets
common_bg = set_bg.intersection(set_bg_2)

# Print the number of common zip codes
print("Number of common BG:", len(common_bg))

Number of common BG: 82259


In [30]:
len(common_bg)/len(list_censusBG)

0.770316333601783

In [31]:
len(common_bg)

82259

In [32]:
len(list_censusBG)-len(common_bg)

24527

In [33]:
BG_list_df2022 = temp2022[temp2022['GEOID'].isin(common_bg)]
BG_list_df2020 = temp2020[temp2020['GEOID'].isin(common_bg)]

In [34]:
print(BG_list_df2022.shape[0])
print(BG_list_df2020.shape[0])

80815
82259


In [218]:
BG_df = pd.concat([BG_list_df2020], ignore_index=True)

In [219]:
BG_df = BG_df.drop_duplicates(subset=['GEOID', 'year'])

In [61]:
list_2020_BG = common_bg

### censusBlockGroupFips

In [212]:
test = df[df['censusBlockGroupFips'].notna()]

In [213]:
print('Total observations available:', test.shape[0])
print('Observations lost:', df.shape[0]-test.shape[0])
print('Percentage NA:', (df.shape[0]-test.shape[0])*100/df.shape[0])
print('Total unique censusBlockGroup:', test.drop_duplicates(subset=['censusBlockGroupFips']).shape[0])

Total observations available: 2447264
Observations lost: 136978
Percentage NA: 5.300509781978623
Total unique censusBlockGroup: 106786


In [214]:
df_geographic_unique = df[['censusBlockGroupFips', 'yearOfLoss']].drop_duplicates()
df_geographic_unique = df_geographic_unique.dropna(subset=['censusBlockGroupFips'])

In [215]:
df_geographic_unique['censusBlockGroupFips'] = [str(int(float(i))) for i in df_geographic_unique['censusBlockGroupFips']]
df_geographic_unique['censusBlockGroupFips'] = [censusBG.zfill(12) for censusBG in df_geographic_unique['censusBlockGroupFips']]

In [216]:
# Define bins and labels for yearOfLoss_1990_2021
bins_1990_2021 = [df_geographic_unique['yearOfLoss'].min(), 2000, 2010, 2020, df_geographic_unique['yearOfLoss'].max() + 1]
labels_1990_2021 = [1990, 2000, 2010, 2020]

df_geographic_unique['yearOfLoss_1990_2021'] = pd.cut(df_geographic_unique['yearOfLoss'], bins=bins_1990_2021, labels=labels_1990_2021, right=False).astype(int)

df_geographic_unique = df_geographic_unique.drop(columns='yearOfLoss')

In [217]:
df_geographic_unique = df_geographic_unique.drop_duplicates()

In [220]:
BG_df.rename(columns={'geometry': 'geometry_BG'}, inplace=True)

In [221]:
BG_df.set_index(['GEOID'], inplace=True)
df_geographic_unique['geometry_BG'] = df_geographic_unique.set_index(['censusBlockGroupFips']).index.map(BG_df['geometry_BG'])

# Find rows with NaN values 
na_rows = df_geographic_unique['geometry_BG'].isna()

BG_df.reset_index(inplace=True)

In [222]:
test = df_geographic_unique.loc[na_rows, ]
print('Total missing:', test[['censusBlockGroupFips']].drop_duplicates().shape[0])
print('Number of censusBlockGroupFips per decade without a matching geometry')
print(test.drop_duplicates(subset=['censusBlockGroupFips', 'yearOfLoss_1990_2021'])['yearOfLoss_1990_2021'].value_counts())

Total missing: 24527
Number of censusBlockGroupFips per decade without a matching geometry
1990    15737
2010    15055
2000    13126
2020     6373
Name: yearOfLoss_1990_2021, dtype: int64


In [47]:
df_test = df_copy[df_copy['censusBlockGroupFips'].notna()]

df_test['censusBlockGroupFips'] = [str(int(float(i))) for i in df_test['censusBlockGroupFips']]
df_test['censusBlockGroupFips'] = [censusBG.zfill(12) for censusBG in df_test['censusBlockGroupFips']]

C:\Users\Asus\AppData\Local\Temp\ipykernel_10632\889359098.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['censusBlockGroupFips'] = [str(int(float(i))) for i in df_test['censusBlockGroupFips']]
C:\Users\Asus\AppData\Local\Temp\ipykernel_10632\889359098.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['censusBlockGroupFips'] = [censusBG.zfill(12) for censusBG in df_test['censusBlockGroupFips']]


In [48]:
print('Value_counts of censusBlockGroupFips per decade:\n', df_test.drop_duplicates(subset=['censusBlockGroupFips', 'yearOfLoss_1990_2021'])['yearOfLoss_1990_2021'].value_counts())

Value_counts of censusBlockGroupFips per decade:
 1990    71925
2010    61534
2000    52227
2020    24273
Name: yearOfLoss_1990_2021, dtype: int64


In [49]:
test.set_index(['censusBlockGroupFips', 'yearOfLoss_1990_2021'], inplace=True)
df_test.set_index(['censusBlockGroupFips', 'yearOfLoss_1990_2021'], inplace=True)

# Select rows of df_copy that have indices found in test
matching_rows = df_test.loc[df_test.index.isin(test.index)]

matching_rows.reset_index(inplace=True)
test.reset_index(inplace=True)
df_test.reset_index(inplace=True)

print(f"There are {matching_rows.shape[0]} rows in df_copy with the same censusBlockGroupFips and yearOfLoss_1990_2021 as in test.")

There are 678942 rows in df_copy with the same censusBlockGroupFips and yearOfLoss_1990_2021 as in test.


In [50]:
matching_rows['yearOfLoss_1990_2021'].value_counts()

1990    229625
2000    219100
2010    185362
2020     44855
Name: yearOfLoss_1990_2021, dtype: int64

### Post 2020, using 2020 shapefiles, checking how yearwise match rate is

In [70]:
df_2020 = df_copy[df_copy['yearOfLoss']>=2020]

In [73]:
df_2020 = df_2020[df_2020['censusBlockGroupFips'].notna()]
df_2020['censusBlockGroupFips'] = [str(int(float(i))) for i in df_2020['censusBlockGroupFips']]
df_2020['censusBlockGroupFips'] = [censusBG.zfill(12) for censusBG in df_2020['censusBlockGroupFips']]

In [75]:
#CensusBG list for NC

list_censusBG = df_2020['censusBlockGroupFips'].dropna().drop_duplicates().tolist()

list_censusBG = [str(int(float(i))) for i in list_censusBG]

# add preceding 0 to make 4-digit zip codes into 5-digit
list_censusBG = [censusBG.zfill(12) for censusBG in list_censusBG]

In [76]:
# Convert both lists to sets
set_bg = set(list_censusBG)
set_bg_2 = set(list_2020_BG)

# Find the intersection of the two sets
common_bg = set_bg.intersection(set_bg_2)

# Print the number of common zip codes
print("Number of common BG:", len(common_bg))

Number of common BG: 17900


In [82]:
df_2020[~df_2020['censusBlockGroupFips'].isin(common_bg)]['yearOfLoss'].value_counts()

2022    18305
2021    13181
2020    12653
2023      716
Name: yearOfLoss, dtype: int64

### Post 2020, using 2010 shapefiles, checking how yearwise match rate is

In [132]:
#CensusBG list for NC

list_censusBG = df_2020['censusBlockGroupFips'].dropna().drop_duplicates().tolist()

list_censusBG = [str(int(float(i))) for i in list_censusBG]

# add preceding 0 to make 4-digit zip codes into 5-digit
list_censusBG = [censusBG.zfill(12) for censusBG in list_censusBG]

In [133]:
# Convert both lists to sets
set_bg = set(list_censusBG)
set_bg_2 = set(list_2010_BG)

# Find the intersection of the two sets
common_bg = set_bg.intersection(set_bg_2)

# Print the number of common zip codes
print("Number of common BG:", len(common_bg))

Number of common BG: 24050


In [135]:
(df_2020[~df_2020['censusBlockGroupFips'].isin(common_bg)]['yearOfLoss'].value_counts()/df_2020['yearOfLoss'].value_counts())*100

2022    0.401749
2021    0.163881
2020    0.176635
2023    1.238787
Name: yearOfLoss, dtype: float64

### Among the census blockgroups that matched with 2020 shapefile, how many of those have the same id in 2010 TIGER shapefile..

In [139]:
non_common_list  = list_2020_BG - list_2010_BG

In [140]:
df_copy_2 = df_copy[df_copy['censusBlockGroupFips'].notna()]
df_copy_2['censusBlockGroupFips'] = [str(int(float(i))) for i in df_copy_2['censusBlockGroupFips']]
df_copy_2['censusBlockGroupFips'] = [censusBG.zfill(12) for censusBG in df_copy_2['censusBlockGroupFips']]

C:\Users\Asus\AppData\Local\Temp\ipykernel_10632\380571103.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_copy_2['censusBlockGroupFips'] = [str(int(float(i))) for i in df_copy_2['censusBlockGroupFips']]
C:\Users\Asus\AppData\Local\Temp\ipykernel_10632\380571103.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_copy_2['censusBlockGroupFips'] = [censusBG.zfill(12) for censusBG in df_copy_2['censusBlockGroupFips']]


In [144]:
len(non_common_list)

416

In [142]:
df_copy_2[df_copy_2['censusBlockGroupFips'].isin(non_common_list)].shape[0]

775

In [143]:
df_copy_2[df_copy_2['censusBlockGroupFips'].isin(non_common_list)]['yearOfLoss'].value_counts()

2022    237
2017    133
2021     76
2020     58
2016     53
2012     45
2023     29
2018     26
2005     23
2019     20
2011     15
2004     10
2015      8
2008      7
2010      6
2003      5
2014      3
2000      3
1993      3
2006      3
1998      2
1996      1
1992      1
1991      1
1990      1
1997      1
1995      1
2002      1
2009      1
2001      1
2007      1
Name: yearOfLoss, dtype: int64

In [224]:
test_non_common = df_copy_2[df_copy_2['censusBlockGroupFips'].isin(non_common_list)]

In [231]:
min(test_non_common['asOfDate'])

'2019-09-19T13:45:58.425Z'

In [232]:
test_non_common.head()

,agricultureStructureIndicator,asOfDate,basementEnclosureCrawlspaceType,policyCount,crsClassificationCode,dateOfLoss,elevatedBuildingIndicator,elevationCertificateIndicator,elevationDifference,baseFloodElevation,ratedFloodZone,houseWorship,locationOfContents,lowestAdjacentGrade,lowestFloorElevation,numberOfFloorsInTheInsuredBuilding,nonProfitIndicator,obstructionType,occupancyType,originalConstructionDate,originalNBDate,amountPaidOnBuildingClaim,amountPaidOnContentsClaim,amountPaidOnIncreasedCostOfComplianceClaim,postFIRMConstructionIndicator,rateMethod,smallBusinessIndicatorBuilding,totalBuildingInsuranceCoverage,totalContentsInsuranceCoverage,yearOfLoss,primaryResidenceIndicator,buildingDamageAmount,buildingDeductibleCode,netBuildingPaymentAmount,buildingPropertyValue,causeOfDamage,condominiumCoverageTypeCode,contentsDamageAmount,contentsDeductibleCode,netContentsPaymentAmount,contentsPropertyValue,disasterAssistanceCoverageRequired,eventDesignationNumber,ficoNumber,floodCharacteristicsIndicator,floodWaterDuration,floodproofedIndicator,floodEvent,iccCoverage,netIccPaymentAmount,nfipRatedCommunityNumber,nfipCommunityNumberCurrent,nfipCommunityName,nonPaymentReasonContents,nonPaymentReasonBuilding,numberOfUnits,buildingReplacementCost,contentsReplacementCost,replacementCostBasis,stateOwnedIndicator,waterDepth,floodZoneCurrent,buildingDescriptionCode,rentalPropertyIndicator,state,reportedCity,reportedZipCode,countyCode,censusTract,censusBlockGroupFips,latitude,longitude,id,yearOfLoss_1990_2021
799,0,2022-11-11T11:43:52.917Z,1.0,1,NaN,2011-06-25T00:00:00.000Z,0,nan,NaN,NaN,X,0,2.0,NaN,NaN,2.0,0,NaN,1.0,2007-01-01T00:00:00.000Z,2011-03-25T00:00:00.000Z,13863.35,0.00,0.0,1,7,0,250000.0,100000.0,2011,1,13269.0,1,13863.35,234086.0,2,N,384.0,1,0.00,0.0,0.0,None,269.0,NaN,0.0,0,Late spring storms,30000.0,0.0,460242.0,460242,UNION COUNTY*,1.0,NaN,1.0,262116.0,0.0,R,0,2.0,X,1.0,0,SD,Currently Unavailable,57049.0,46127.0,4.612702e+10,461270203012,42.5,-96.5,80f94523-7dc7-4b70-b9c8-fdea8d794b24,2010
876,0,2022-12-30T15:07:53.713Z,NaN,1,NaN,2005-08-29T00:00:00.000Z,0,nan,NaN,NaN,AE,0,3.0,NaN,NaN,1.0,0,NaN,2.0,1970-01-01T00:00:00.000Z,1990-06-08T00:00:00.000Z,NaN,NaN,NaN,0,1,0,71700.0,0.0,2005,0,NaN,1,0.00,NaN,1,N,NaN,0,0.00,NaN,0.0,None,654.0,NaN,0.0,0,Hurricane Katrina,30000.0,0.0,220139.0,220139,PLAQUEMINES PARISH*,NaN,98.0,1.0,NaN,NaN,A,0,0.0,AE,1.0,0,LA,Currently Unavailable,70037.0,22075.0,2.207505e+10,220750502021,29.9,-90.0,8ef9e023-8d72-4cd8-898f-5389f2990456,2000
5795,0,2022-11-18T18:55:09.397Z,NaN,1,5.0,2017-08-29T00:00:00.000Z,0,nan,NaN,NaN,X,0,NaN,NaN,NaN,1.0,0,NaN,1.0,2006-07-01T00:00:00.000Z,2013-03-27T00:00:00.000Z,200000.00,56023.16,0.0,1,7,0,200000.0,80000.0,2017,1,231381.0,F,200000.00,667662.0,2,N,57273.0,F,56023.16,80000.0,0.0,None,682.0,NaN,0.0,0,Hurricane Harvey,30000.0,0.0,480296.0,480296,"HOUSTON, CITY OF",NaN,NaN,1.0,699655.0,0.0,A,0,1.0,X,1.0,0,TX,Currently Unavailable,77345.0,48201.0,4.820125e+10,482012509024,30.0,-95.1,bec9d646-f589-409f-beae-d9551839377f,2010
7008,0,2022-12-01T15:18:56.655Z,NaN,1,NaN,2008-09-13T00:00:00.000Z,1,nan,-9.0,16.0,V20,0,7.0,7.0,7.0,3.0,0,24.0,1.0,1986-01-01T00:00:00.000Z,1988-03-01T00:00:00.000Z,19287.59,0.00,0.0,1,2,0,250000.0,97800.0,2008,0,9944.0,0,9444.43,354000.0,1,N,NaN,0,0.00,NaN,0.0,None,668.0,NaN,0.0,0,Hurricane Ike,30000.0,0.0,485469.0,485469,"GALVESTON, CITY OF",97.0,NaN,0.0,426000.0,NaN,A,0,7.0,VE,1.0,0,TX,Currently Unavailable,77554.0,48167.0,4.816773e+10,481677261021,29.2,-95.0,caa3b0bc-d4fb-42f1-8c4f-fc0eeae69f56,2000
7747,0,2022-11-15T13:35:49.296Z,4.0,1,6.0,2012-10-31T00:00:00.000Z,0,nan,-3.0,6.7,AE,0,NaN,4.8,3.5,3.0,0,80.0,1.0,1989-01-01T00:00:00.000Z,1998-03-27T00:00:00.000Z,10676.90,0.00,0.0,1,2,0,250000.0,26300.0,2012,1,10459.0,1,10676.90,254849.0,1,N,749.0,1,0.00,0.0,0.0,None,305.0,NaN,0.0,0,Hurricane Sandy,30000.0,0.0,340393.0,340393,"STAFFORD, TOWNSHIP OF",1.0,NaN,1.0,292086.0,0.0,R,0,0.0,VE,1.0,0,NJ,Currently Unavailable,8050.0,34029.0,3.402974e+10,340297351052